## Compare read and write speeds of numpy and pandas

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from random import choice
from src.localdataset.dataset import LocalDataset

In [3]:
ds = LocalDataset()

In [14]:
# store gids and raw np arrays into feather file
pd.DataFrame({"gid": gids, "data":arrays}).to_feather(f"/media/HDD01/mbck/dataSet/raw/total_dataset.ft")

In [12]:
read_data = pd.read_feather("/media/HDD01/mbck/dataSet/raw/total_dataset.ft")

compare read speeds

In [24]:
%timeit read_data[read_data["gid"] == "5b028e16-312a-46bf-913e-65dfbb343470"].data.to_numpy()

277 µs ± 6.78 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
from src.localdataset.tools import regenerate_mission_windows_from_tag

gidss = read_data["gid"]

In [7]:
%timeit regenerate_mission_windows_from_tag(read_data[read_data["gid"] == choice(gidss)].data.to_numpy(), {"timestamp":50}, 300)

285 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


use python dict to store data

In [13]:
dict_data = {d.gid:d.data for _, d in read_data.iterrows()}

In [13]:
%timeit regenerate_mission_windows_from_tag(dict_data[choice(gidss)], {"timestamp":50}, 300)

2.44 µs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
%timeit choice(gidss)

2.06 µs ± 18 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


numpy file

In [25]:
np.savez("/media/HDD01/mbck/dataSet/raw/total_dataset.npz", **data)

In [ ]:
np_read_data = lambda(x: np.load(x))

In [30]:
%timeit np_read_data["5b028e16-312a-46bf-913e-65dfbb343470"]

2.42 ms ± 54.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Demostrate results

In [4]:
demo = ds.get_data(preprocessing="window_300")

In [5]:
train_demo = demo.train
train_choices = list(range(len(train_demo)))

In [10]:
%timeit train_demo.get_by_index(choice(train_choices)).data

55.1 µs ± 418 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


this results seam to be very high :( -> check which code elements take how long

In [12]:
assert_choices = range(500)

In [13]:
%timeit assert choice(assert_choices) < len(assert_choices), f"data contains only {len(self.__data)} elements, you tried to acces index {index}!"

342 ns ± 1.59 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [4]:
demo_mission = pd.read_feather("/media/HDD01/mbck/dataSet/dataset_info/V7.ft").merge(pd.read_feather("/media/HDD01/mbck/dataSet/preprocessing/window_300/info.ft"), left_on="gid", right_on="gid")

In [21]:
demo_mission["file_name"] = demo_mission["gid"]
demo_mission["id"] = demo_mission["gid"]

In [9]:
demo_mission_choices = list(demo_mission.index)

In [11]:
%timeit demo_mission.iloc[choice(demo_mission_choices)]

26.4 µs ± 876 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [33]:
from src.localdataset.dataset import Mission
demo_dataframe_entity = demo_mission.iloc[choice(demo_mission_choices)]

In [34]:
%timeit Mission.from_mission_data(demo_dataframe_entity, lambda x:x, Path(""), regenerate_mission_windows_from_tag, [300], dict_data[demo_dataframe_entity.gid])

19.5 µs ± 91 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [30]:
timing_gid = demo_mission.iloc[choice(demo_mission_choices)]
timing_mission = Mission.from_mission_data(timing_gid, lambda x:x, Path(""), regenerate_mission_windows_from_tag, [300], dict_data[timing_gid.gid])

In [31]:
%timeit timing_mission.data

448 ns ± 20.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Conclusion

Using pandas with feather is faster then using numpy and savez